# Wrangle and Analyze Data Project: We Rate Dogs
### By Bakut S Bonat

## INTRODUCTION


This project was carried out as result of the lesson; Data Wrangling i took and a periquiste for gruaduating the udacity Data Analyst Nano-degree. 

The dataset to be wrangled, analyzed and visualized is the tweet archive of Twitter user @dog_rates, also known as WeRateDogs. WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10. The numerators, though? Almost always greater than 10. 11/10, 12/10, 13/10, etc. Why? Because "they're good dogs Brent." WeRateDogs has over 4 million followers and has received international media coverage.

WeRateDogs downloaded their Twitter archive and sent it to Udacity via email exclusively for use in this project. This archive contains basic tweet data (tweet ID, timestamp, text, etc.) for all 5000+ of their tweets as they stood on August 1, 2017.

## Data Gathering

1. **Twitter archive file:** downloadable file in the Resources tab provided by Udacity named as: twitter_archive_enhanced.csv

In [1]:
#Import all packages needed
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import tweepy
import json


In [2]:
#Read CSV file
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
twitter_archive.sort_values('timestamp')
twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [4]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

2. **The tweet image predictions,** i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file (image_predictions.tsv) is hosted on Udacity's servers and should be downloaded programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

In [5]:
#URL downloaded programatically
url= "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(url)

with open('image-predictions.tsv', mode = 'wb') as file:
    file.write(response.content)
    
#Read TSV file
image_prediction = pd.read_csv('image-predictions.tsv', sep='\t' )

In [6]:
image_prediction.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


In [7]:
#https://stackoverflow.com/questions/28384588/twitter-api-get-tweets-with-specific-id
auth = tweepy.OAuthHandler('5Uur0mo4ol2kB8yhtZ1VxXS0u', 'h8E7fSpXWiMoBel7G1ZOAeu4Mgru0v0MtxH5ehYE1RKM89SiBH')
auth.set_access_token('303562412-ct9aNnU0FQR0UKJVn1i1W3Y8omqSewiQWUcRaygB', 'D3qslrbdOU5fqTOp951kOIuZbkeTPBodnjNYoEGFR63Ft')
api = tweepy.API(auth, 
                 parser = tweepy.parsers.JSONParser(), 
                 wait_on_rate_limit = True, 
                 wait_on_rate_limit_notify = True)

3.  **Twitter API & JSON:** Each tweet's retweet count and favorite ("like") count at minimum, and any additional data you find interesting. Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count.

In [8]:
#Download Tweepy status object based on Tweet ID and store in list
list_of_tweets = []
# Tweets that can't be found are saved in the list below:
cant_find_tweets_for_those_ids = []
for tweet_id in twitter_archive['tweet_id']:   
    try:
        list_of_tweets.append(api.get_status(tweet_id))
    except Exception as e:
        cant_find_tweets_for_those_ids.append(tweet_id)

Rate limit reached. Sleeping for: 730
Rate limit reached. Sleeping for: 727


In [9]:
print("The list of tweets" ,len(list_of_tweets))
print("The list of tweets no found" , len(cant_find_tweets_for_those_ids))

The list of tweets 2339
The list of tweets no found 17


In [10]:
#Then in this code block we isolate the json part of each tweepy 
#status object that we have downloaded and we add them all into a list

my_list_of_dicts = []
for each_json_tweet in list_of_tweets:
    my_list_of_dicts.append(each_json_tweet)

In [11]:
#we write this list into a txt file:

with open('tweet_json.txt', 'w') as file:
        file.write(json.dumps(my_list_of_dicts, indent=4))

In [12]:
#identify information of interest from JSON dictionaries in txt file
#and put it in a dataframe called tweet JSON
my_demo_list = []
with open('tweet_json.txt', encoding='utf-8') as json_file:  
    all_data = json.load(json_file)
    for each_dictionary in all_data:
        tweet_id = each_dictionary['id']
        whole_tweet = each_dictionary['text']
        only_url = whole_tweet[whole_tweet.find('https'):]
        favorite_count = each_dictionary['favorite_count']
        retweet_count = each_dictionary['retweet_count']
        followers_count = each_dictionary['user']['followers_count']
        friends_count = each_dictionary['user']['friends_count']
        whole_source = each_dictionary['source']
        only_device = whole_source[whole_source.find('rel="nofollow">') + 15:-4]
        source = only_device
        retweeted_status = each_dictionary['retweeted_status'] = each_dictionary.get('retweeted_status', 'Original tweet')
        if retweeted_status == 'Original tweet':
            url = only_url
        else:
            retweeted_status = 'This is a retweet'
            url = 'This is a retweet'

        my_demo_list.append({'tweet_id': str(tweet_id),
                             'favorite_count': int(favorite_count),
                             'retweet_count': int(retweet_count),
                             'followers_count': int(followers_count),
                             'friends_count': int(friends_count),
                             'url': url,
                             'source': source,
                             'retweeted_status': retweeted_status,
                            })
        tweet_json = pd.DataFrame(my_demo_list, columns = ['tweet_id', 'favorite_count','retweet_count', 
                                                           'followers_count', 'friends_count','source', 
                                                           'retweeted_status', 'url'])

In [13]:
tweet_json.head()


,tweet_id,favorite_count,retweet_count,followers_count,friends_count,source,retweeted_status,url
0,892420643555336193,37675,8213,7935796,12,Twitter for iPhone,Original tweet,https://t.co/MgUWQ76dJU
1,892177421306343426,32359,6074,7935796,12,Twitter for iPhone,Original tweet,https://t.co/aQFSeaCu9L
2,891815181378084864,24377,4015,7935796,12,Twitter for iPhone,Original tweet,https://t.co/r0YlrsGCgy
3,891689557279858688,40994,8369,7935796,12,Twitter for iPhone,Original tweet,https://t.co/tD36da7qLQ
4,891327558926688256,39195,9072,7935796,12,Twitter for iPhone,Original tweet,https://t.co/0g0KMIVXZ3


## Assessing Data

#### Visual Assessment

In [14]:
twitter_archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [15]:
image_prediction.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [16]:
tweet_json

,tweet_id,favorite_count,retweet_count,followers_count,friends_count,source,retweeted_status,url
0,892420643555336193,37675,8213,7935796,12,Twitter for iPhone,Original tweet,https://t.co/MgUWQ76dJU
1,892177421306343426,32359,6074,7935796,12,Twitter for iPhone,Original tweet,https://t.co/aQFSeaCu9L
2,891815181378084864,24377,4015,7935796,12,Twitter for iPhone,Original tweet,https://t.co/r0YlrsGCgy
3,891689557279858688,40994,8369,7935796,12,Twitter for iPhone,Original tweet,https://t.co/tD36da7qLQ
4,891327558926688256,39195,9072,7935796,12,Twitter for iPhone,Original tweet,https://t.co/0g0KMIVXZ3
5,891087950875897856,19707,3010,7935796,12,Twitter for iPhone,Original tweet,https://t.co/xx5cilW0Dd
6,890971913173991426,11521,1989,7935796,12,Twitter for iPhone,Original tweet,https://t.co/MV01Q820LT
7,890729181411237888,63539,18242,7935796,12,Twitter for iPhone,Original tweet,https://t.co/hrcFOGi12V
8,890609185150312448,27092,4132,7935796,12,Twitter for iPhone,Original tweet,https://t.co/UkrdQyoYxV
9,890240255349198849,31061,7132,7935796,12,Twitter for iPhone,Original tweet,https://t.co/l3TSS3o2M0


#### Programmatic Assessment

The use of panda functions/methods to assess data

In [17]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [18]:
sum(twitter_archive['tweet_id'].duplicated())

0

In [19]:
twitter_archive.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [20]:
print(twitter_archive.loc[twitter_archive.rating_numerator == 420, 'text'])
print(twitter_archive.loc[twitter_archive.rating_numerator == 2, 'text'])

188     @dhmontgomery We also gave snoop dogg a 420/10...
2074    After so many requests... here you go.\n\nGood...
Name: text, dtype: object
1761    Exotic pup here. Tail long af. Throat looks sw...
1764    This is Crystal. She's a shitty fireman. No se...
1920    This is Henry. He's a shit dog. Short pointy e...
2079    Scary dog here. Too many legs. Extra tail. Not...
2237    This lil pup is Oliver. Hops around. Has wings...
2246    This is Tedrick. He lives on the edge. Needs s...
2310    Unfamiliar with this breed. Ears pointy af. Wo...
2326    This is quite the dog. Gets really excited whe...
2349    This is an odd dog. Hard on the outside but lo...
Name: text, dtype: object


In [21]:
#print whole text in order to verify numerators and denominators
print(twitter_archive['text'][2349])
print(twitter_archive['text'][2074])

This is an odd dog. Hard on the outside but loving on the inside. Petting still fun. Doesn't play catch well. 2/10 https://t.co/v5A4vzSDdc
After so many requests... here you go.

Good dogg. 420/10 https://t.co/yfAAo1gdeY


In [22]:
twitter_archive.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [23]:
print(twitter_archive.loc[twitter_archive.rating_denominator == 11, 'text'])
print(twitter_archive.loc[twitter_archive.rating_denominator == 120, 'text'])

784     RT @dog_rates: After so many requests, this is...
1068    After so many requests, this is Bretagne. She ...
1662    This is Darrel. He just robbed a 7/11 and is i...
Name: text, dtype: object
1779    IT'S PUPPERGEDDON. Total of 144/120 ...I think...
Name: text, dtype: object


In [24]:
#print whole text in order to verify numerators and denominators
print(twitter_archive['text'][784])
print(twitter_archive['text'][1779])

RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…
IT'S PUPPERGEDDON. Total of 144/120 ...I think https://t.co/ZanVtAtvIq


In [25]:
image_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [26]:
sum(image_prediction.tweet_id.duplicated())

0

In [27]:
sum(image_prediction.jpg_url.duplicated())

66

In [28]:
tweet_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2339 entries, 0 to 2338
Data columns (total 8 columns):
tweet_id            2339 non-null object
favorite_count      2339 non-null int64
retweet_count       2339 non-null int64
followers_count     2339 non-null int64
friends_count       2339 non-null int64
source              2339 non-null object
retweeted_status    2339 non-null object
url                 2339 non-null object
dtypes: int64(4), object(4)
memory usage: 146.3+ KB


### Data Quality

Completeness, Validity, accuracy and consistency.

### twitter_archive

1. Keep originals ratings(no retweets) that have images.
2. Separate timestamp into day-month-year(3 different coloumns).
3. Delete columns that won't be used for analysis.
4. Correct denominators other than 10(manually and programatically).
5. correct numerators.

### image_prediction

1. Delete columns that won't be used.
2. Drop 66 jpg_url duplicated.

### tweet_json

1. Keep original tweets only.
2. 1. change tweet_id to type int64 in order to merge with the other 2 tables.

### Tidiness

1. Erroneous datatypes(doggo, floofer, pupper and puppo columns)
2. All tables should be part of one dataset.

### Cleaning Data


In [118]:
twitter_archive_clean = twitter_archive.copy()
image_prediction_clean = image_prediction.copy()
tweet_json_clean = tweet_json.copy()

1. **Twitter Archive (Define)-** keep original ratings(no retweets) that have images
from twitter_archive.info() there are 181 values in both retweeted_status_id and retweeted_status_user_id. delete the retweets, once i merge twitter_archive and image_predictions, i will only keep the ones with images

In [51]:
#CODE: Delete retweets by filtering the NaN of retweeted_status_user_id
twitter_archive_clean = twitter_archive_clean[pd.isnull(twitter_archive_clean['retweeted_status_user_id'])]

#TEST
print(sum(twitter_archive_clean.retweeted_status_user_id.value_counts()))

0


2. **twitter Archive(Define) -** Separate timestamp into day-month-year(in 3 different columns)
First convert timestamp to datetime. Then extract year, month and day to new columns. finally drop timestamp column.

In [120]:
#CODE: convert timestamp to datetime
twitter_archive_clean['timestamp'] = pd.to_datetime(twitter_archive_clean['timestamp'])

#CODE: extract year, month and day to new columns
twitter_archive_clean['year']= twitter_archive_clean['timestamp'].dt.year
twitter_archive_clean['month']= twitter_archive_clean['timestamp'].dt.month
twitter_archive_clean['day']= twitter_archive_clean['timestamp'].dt.day

In [121]:
#COEDE: drop timestamp column
twitter_archive_clean = twitter_archive_clean.drop('timestamp', 1)

In [122]:
#TEST:
list(twitter_archive_clean)

['tweet_id',
 'text',
 'rating_numerator',
 'rating_denominator',
 'name',
 'dogs_stage',
 'year',
 'month',
 'day']

3. **Twitter archive(Define)**- delete columns that won't be used for analysis

In [ ]:
#CODE: get the column names of twitter_archive_clean
print(list(twitter_archive_clean))

In [ ]:
#CODE: delete columns not needed
twitter_archive_clean = twitter_archive_clean.drop(['source','in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','expanded_urls'], 1)

In [ ]:
#TEST
list(twitter_archive_clean)

### **Tidiness**

 Erroneous datatypes (doggo, floofer, pupper and puppo columns)
 
**Define:-**
Melt the doggo, floofer, pupper and puppo columns to dogs and dogs_stage column. Then drop dogs. Sort by dogs_stage in order to then drop duplicated based on tweet_id except for the last occurrence.

In [123]:
#CODE: Melt the doggo, floofer, pupper and puppo columns to dogs and dogs_stage column

twitter_archive_clean = pd.melt(twitter_archive_clean, id_vars=['tweet_id',                                          
                                                                'timestamp',
                                                                'text',
                                                                'rating_numerator',
                                                                'rating_denominator',
                                                                'name'],
                               var_name='dogs', value_name='dogs_stage')

#CODE: drop dogs
twitter_archive_clean = twitter_archive_clean.drop('dogs', 1)

#CODE: Sort by dogs_stage then drop duplicated based on tweet_id except the last occurrence
twitter_archive_clean = twitter_archive_clean.sort_values('dogs_stage').drop_duplicates(subset='tweet_id', 
                                                                                        keep='last')

KeyError: 'timestamp'

In [124]:
#TEST
list(twitter_archive_clean)

['tweet_id',
 'text',
 'rating_numerator',
 'rating_denominator',
 'name',
 'dogs_stage',
 'year',
 'month',
 'day']

4. **Twitter Archives(Define)-** Correct Denominators

**Manually:-** Five tweets with denominator not equal to 10 for special circunstances. Update both numerators and denominators when necessary. Delete other five tweets because they do not have actual ratings.

In [93]:
#CODE: Update both numerators and denominators
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 740373189193256964), 'rating_numerator'] = 14
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 740373189193256964), 'rating_denominator'] = 10

twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 682962037429899265), 'rating_numerator'] = 10
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 682962037429899265), 'rating_denominator'] = 10

twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 666287406224695296), 'rating_numerator'] = 9
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 666287406224695296), 'rating_denominator'] = 10

twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 722974582966214656), 'rating_numerator'] = 13
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 722974582966214656), 'rating_denominator'] = 10

twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 716439118184652801), 'rating_numerator'] = 13.5
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 716439118184652801), 'rating_denominator'] = 10

In [94]:
#CODE: Delete five tweets with no actual ratings
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['tweet_id'] != 832088576586297345]
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['tweet_id'] != 810984652412424192]
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['tweet_id'] != 682808988178739200]
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['tweet_id'] != 835246439529840640]
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['tweet_id'] != 686035780142297088]

In [95]:
#CODE: Left only the group dogs for programatically clean
with pd.option_context('max_colwidth', 200):
    display(twitter_archive_clean[twitter_archive_clean['rating_denominator'] != 10][['tweet_id',
                                                                                      'text',
                                                                                      'rating_numerator',
                                                                                      'rating_denominator']])

,tweet_id,text,rating_numerator,rating_denominator
363,820690176645140481,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84.0,70.0
733,758467244762497024,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,165.0,150.0
942,731156023742988288,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,204.0,170.0
1050,713900603437621249,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1,99.0,90.0
1075,710658690886586372,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12,80.0,80.0
1095,709198395643068416,"From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK",45.0,50.0
1172,704054845121142784,Here is a whole flock of puppers. 60/50 I'll take the lot https://t.co/9dpcw6MdWa,60.0,50.0
1254,697463031882764288,Happy Wednesday here's a bucket of pups. 44/40 would pet all at once https://t.co/HppvrYuamZ,44.0,40.0
1455,684225744407494656,"Two sneaky puppers were not initially seen, moving the rating to 143/130. Please forgive us. Thank you https://t.co/kRK51Y5ac3",143.0,130.0
1456,684222868335505415,Someone help the girl is being mugged. Several are distracting her while two steal her shoes. Clever puppers 121/110 https://t.co/1zfnTJLt55,121.0,110.0


**Programatically:-** 

These tweets with denominator not equal to 10 are multiple dogs. For example, tweet_id 713900603437621000 has numerator and denominators 99/90 because there are 9 dogs in the picture https://t.co/mpvaVxKmc1.

In [96]:
#CODE: Create a new column with rating in float type to avoid converting all int column to float
twitter_archive_clean['rating'] = 10 * twitter_archive_clean['rating_numerator'] / twitter_archive_clean['rating_denominator'].astype(float)

twitter_archive_clean.head()

,tweet_id,text,rating_numerator,rating_denominator,name,dogs_stage,year,month,day,rating
0,892420643555336193,This is Phineas. He's a mystical boy. Only eve...,13.0,10.0,Phineas,NaN,2017,8,1,13.0
1,892177421306343426,This is Tilly. She's just checking pup on you....,13.0,10.0,Tilly,NaN,2017,8,1,13.0
2,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,12.0,10.0,Archie,NaN,2017,7,31,12.0
3,891689557279858688,This is Darla. She commenced a snooze mid meal...,13.0,10.0,Darla,NaN,2017,7,30,13.0
4,891327558926688256,This is Franklin. He would like you to stop ca...,12.0,10.0,Franklin,NaN,2017,7,29,12.0


6. **Twitter Archive:-** Correct Numerators

In [97]:
#CODE
twitter_archive_clean[['rating_numerator', 'rating_denominator']]= twitter_archive_clean[['rating_numerator', 'rating_denominator']].astype(float)

twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23870 entries, 0 to 23924
Data columns (total 10 columns):
tweet_id              23870 non-null int64
text                  23870 non-null object
rating_numerator      23870 non-null float64
rating_denominator    23870 non-null float64
name                  23870 non-null object
dogs_stage            13114 non-null object
year                  23870 non-null int64
month                 23870 non-null int64
day                   23870 non-null int64
rating                23870 non-null float64
dtypes: float64(3), int64(4), object(3)
memory usage: 2.0+ MB


In [98]:
#CODE: First change numerator and denominators type int to float to allow decimals 
twitter_archive_clean[['rating_numerator', 'rating_denominator']] = twitter_archive_clean[['rating_numerator','rating_denominator']].astype(float)

#Update numerators

twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 883482846933004288), 'rating_numerator'] = 13.5
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 786709082849828864), 'rating_numerator'] = 9.75
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 778027034220126208), 'rating_numerator'] = 11.27
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 681340665377193984), 'rating_numerator'] = 9.5
twitter_archive_clean.loc[(twitter_archive_clean.tweet_id == 680494726643068929), 'rating_numerator'] = 11.26


with pd.option_context('max_colwidth', 200):
    display(twitter_archive_clean[twitter_archive_clean['text'].str.contains(r"(\d+\.\d*\/\d+)")]
            [['tweet_id', 'text', 'rating_numerator', 'rating_denominator']])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


,tweet_id,text,rating_numerator,rating_denominator
42,883482846933004288,"This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",13.50,10.0
558,786709082849828864,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",9.75,10.0
617,778027034220126208,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,11.27,10.0
1510,681340665377193984,I've been told there's a slight possibility he's checking his mirror. We'll bump to 9.5/10. Still a menace,9.50,10.0
1533,680494726643068929,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,11.26,10.0
2217,883482846933004288,"This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",13.50,10.0
2733,786709082849828864,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",9.75,10.0
2792,778027034220126208,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,11.27,10.0
3685,681340665377193984,I've been told there's a slight possibility he's checking his mirror. We'll bump to 9.5/10. Still a menace,9.50,10.0
3708,680494726643068929,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,11.26,10.0


**Image Prediction:-** 
1. Delete columns that won't be used; Create a function where i keep the first true prediction along the confidence level as new columns.

In [99]:
#CODE: The first true prediction (p1, p2 or p3) will be store in these lists
dog_type = []
confidence_list = []

#create a function with nested if to capture the dog type and confidence level
# from the first 'true' prediction
def image(image_prediction_clean):
    if image_prediction_clean['p1_dog'] == True:
        dog_type.append(image_prediction_clean['p1'])
        confidence_list.append(image_prediction_clean['p1_conf'])
    elif image_prediction_clean['p2_dog'] == True:
        dog_type.append(image_prediction_clean['p2'])
        confidence_list.append(image_prediction_clean['p2_conf'])
    elif image_prediction_clean['p3_dog'] == True:
        dog_type.append(image_prediction_clean['p3'])
        confidence_list.append(image_prediction_clean['p3_conf'])
    else:
        dog_type.append('Error')
        confidence_list.append('Error')

#series objects having index the image_prediction_clean column.        
image_prediction_clean.apply(image, axis=1)

#create new columns
image_prediction_clean['dog_type'] = dog_type
image_prediction_clean['confidence_list'] = confidence_list

KeyError: ('p1_dog', 'occurred at index 0')

In [100]:
#drop rows that has prediction_list 'error'
image_prediction_clean = image_prediction_clean[image_prediction_clean['dog_type'] != 'Error']


In [101]:
#TEST 
image_prediction_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1691 entries, 0 to 2073
Data columns (total 4 columns):
tweet_id           1691 non-null int64
jpg_url            1691 non-null object
dog_type           1691 non-null object
confidence_list    1691 non-null object
dtypes: int64(1), object(3)
memory usage: 66.1+ KB


2. **Image Prediction:-** Drop duplicated 66 jpg_url

In [102]:
#Delet duplicated jpg_url
image_prediction_clean= image_prediction_clean.drop_duplicates(subset=['jpg_url'], keep='last')

In [103]:
sum(image_prediction_clean['jpg_url'].duplicated())

0

3. **Image Prediction:-** Delete columns that won't be used for analysis

In [104]:
#print list of image_prediction columns
print(list(image_prediction_clean))

#Delete columns
image_prediction_clean = image_prediction_clean.drop(['img_num', 'p1', 
                                                      'p1_conf', 'p1_dog', 
                                                      'p2', 'p2_conf', 
                                                      'p2_dog', 'p3', 
                                                      'p3_conf', 
                                                      'p3_dog'], 1)

['tweet_id', 'jpg_url', 'dog_type', 'confidence_list']


KeyError: "['img_num' 'p1' 'p1_conf' 'p1_dog' 'p2' 'p2_conf' 'p2_dog' 'p3' 'p3_conf'\n 'p3_dog'] not found in axis"

In [105]:
list(image_prediction_clean)

['tweet_id', 'jpg_url', 'dog_type', 'confidence_list']

1. **Tweet Json:-** keep original tweets

In [106]:
tweet_json_clean= tweet_json_clean[tweet_json_clean['retweeted_status'] == 'Original tweet']

In [107]:
tweet_json_clean['retweeted_status'].value_counts()

Original tweet    2172
Name: retweeted_status, dtype: int64

2.**Tweet Json(Define):-** Change tweet_id to type int64 in order to merge with the others 2 tables

In [108]:
#CODE: Change tweet_id from str to int
tweet_json_clean['tweet_id']= tweet_json_clean['tweet_id'].astype(int)

In [109]:
#TEST
tweet_json_clean['tweet_id'].dtypes

dtype('int64')

### Tidiness

In [110]:
#CODE: create a new dataframe that merge twiiter_archive_clean and image_prediction_clean
df_ratedogs1= pd.merge(twitter_archive_clean,
                       image_prediction_clean,
                       how = 'left', on = ['tweet_id'])

In [111]:
#CODE: keep rows that have pictures (jpg_url)
df_ratedogs1= df_ratedogs1[df_ratedogs1['jpg_url'].notnull()]

df_ratedogs1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17875 entries, 1 to 23869
Data columns (total 13 columns):
tweet_id              17875 non-null int64
text                  17875 non-null object
rating_numerator      17875 non-null float64
rating_denominator    17875 non-null float64
name                  17875 non-null object
dogs_stage            9790 non-null object
year                  17875 non-null int64
month                 17875 non-null int64
day                   17875 non-null int64
rating                17875 non-null float64
jpg_url               17875 non-null object
dog_type              17875 non-null object
confidence_list       17875 non-null object
dtypes: float64(3), int64(4), object(6)
memory usage: 1.9+ MB


In [112]:
#CODE: Create a new dataframe that merge df_twitter and tweet_json_clean
df_ratedogs= pd.merge(df_ratedogs1, tweet_json_clean,
                     how = 'left', on = ['tweet_id'])

In [113]:
#TEST
df_ratedogs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17875 entries, 0 to 17874
Data columns (total 20 columns):
tweet_id              17875 non-null int64
text                  17875 non-null object
rating_numerator      17875 non-null float64
rating_denominator    17875 non-null float64
name                  17875 non-null object
dogs_stage            9790 non-null object
year                  17875 non-null int64
month                 17875 non-null int64
day                   17875 non-null int64
rating                17875 non-null float64
jpg_url               17875 non-null object
dog_type              17875 non-null object
confidence_list       17875 non-null object
favorite_count        17842 non-null float64
retweet_count         17842 non-null float64
followers_count       17842 non-null float64
friends_count         17842 non-null float64
source                17842 non-null object
retweeted_status      17842 non-null object
url                   17842 non-null object
dtypes: f

In [114]:
#TEST
df_ratedogs['rating_numerator'].value_counts()

12.00     4444
10.00     3894
11.00     3762
13.00     2332
9.00      1463
8.00       748
7.00       341
14.00      242
6.00       176
5.00       143
4.00        66
3.00        55
2.00        22
13.50       22
88.00       11
45.00       11
44.00       11
9.75        11
121.00      11
11.26       11
60.00       11
80.00       11
11.27       11
165.00      11
84.00       11
0.00        11
99.00       11
144.00      11
143.00      11
Name: rating_numerator, dtype: int64

In [115]:
#TEST
df_ratedogs['rating_denominator'].value_counts()

10.0     17754
80.0        22
50.0        22
110.0       11
120.0       11
150.0       11
70.0        11
90.0        11
130.0       11
40.0        11
Name: rating_denominator, dtype: int64

### Storing

In [116]:
#store the clean Dataframe in csv file
df_ratedogs.to_csv('twitter_archive_master.csv',
                  index= False, encoding = 'utf-8' )

### Insights and Visualizations

INSIGHT ONE
1. Descriptive Statistical summary of the dataset

In [117]:
df= pd.read_csv('twitter_archive_master.csv')
df.describe()

,tweet_id,rating_numerator,rating_denominator,year,month,day,rating,confidence_list,favorite_count,retweet_count,followers_count,friends_count
count,1.787500e+04,17875.000000,17875.000000,17875.000000,17875.000000,17875.000000,17875.000000,17875.000000,17842.000000,17842.000000,1.784200e+04,17842.0
mean,7.383904e+17,11.389711,10.529231,2015.880615,6.980923,15.974154,10.803865,0.550108,8342.437731,2416.954994,7.935808e+06,12.0
std,6.829169e+16,8.012308,6.996672,0.698519,4.112865,8.830716,1.783511,0.299985,12229.629194,4342.915178,7.482268e+01,0.0
min,6.660209e+17,0.000000,10.000000,2015.000000,1.000000,1.000000,0.000000,0.000010,77.000000,11.000000,7.935770e+06,12.0
25%,6.769579e+17,10.000000,10.000000,2015.000000,3.000000,8.000000,10.000000,0.302200,1971.000000,582.000000,7.935778e+06,12.0
50%,7.106587e+17,11.000000,10.000000,2016.000000,7.000000,16.000000,11.000000,0.550506,3934.500000,1259.500000,7.935798e+06,12.0
75%,7.931204e+17,12.000000,10.000000,2016.000000,11.000000,24.000000,12.000000,0.822936,10251.000000,2772.000000,7.935833e+06,12.0
max,8.921774e+17,165.000000,150.000000,2017.000000,12.000000,31.000000,14.000000,0.999956,162654.000000,82765.000000,7.936462e+06,12.0


    *  Notes
1. Mean rating of 10.86 with an outlier of 75 which is the maximum rating.



2. Mean favorite_count of 8346, a maximum value and minimum value  of 162710 76 respectively.

**Question:-**  what are the actual maximum and minimum favorite tweets from WeRateDogs?

In [ ]:
#maximum favorite tweet count from we rate dogs
df[df['favorite_count'] == 162710 ]

In [ ]:
#minimum Favorite tweet count  from the we rate dogs
df[df['favorite_count'] == 76]

3. Mean retweet_count of 2417, a maximum value and minimum value of 82799 and 11 respectively.

**Question:-** what are the actual maximum and minimum retweet_count from we rate Dogs 

In [ ]:
#maximum retweet_count from the we rate Dogs tweets
df[df['retweet_count'] == 82799]

In [ ]:
#minimum retweet_count from the we rate Dogs tweets
df[df['retweet_count'] == 11]

INSIGHT TWO
2. Visuals comparing the Favorite and Retweet Counts

In [ ]:
sns.set_style('whitegrid')
g = sns.jointplot(x="retweet_count", 
                  y="favorite_count", 
                  data=df, 
                  color="blue",
                  kind="kde", 
                  size=10)
g.plot_joint(plt.scatter,  
             c="black",
             s=80, 
             linewidth=1, 
             marker="+",
             alpha=0.45)
g.set_axis_labels("Retweet Count", "Favorite Count",fontsize=12)

g.fig.subplots_adjust(top=0.9)
plt.suptitle('Correlation between the retweet & favorite counts',
                 fontsize=16)

INSIGHT THREE

**Most used twitter source**

In [ ]:
df['source'].value_counts().plot('barh', figsize=(11,5), title='Most used Twitter source').set_xlabel("Number of Tweets")
plt.savefig('twitter_source')

**REFERENCES**
1. https://stackoverflow.com/questions/3768895/how-to-make-a-class-json-serializable?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
2. https://stackoverflow.com/questions/1024847/add-new-keys-to-a-dictionary
https://stackoverflow.com/questions/27900451/convert-tweepy-status-object-into-json?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
3. https://stackoverflow.com/questions/17157753/get-the-error-code-from-tweepy-exception-instance?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
4. https://stackoverflow.com/questions/16616141/deleting-all-columns-except-a-few-python-pandas?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
5. https://stackoverflow.com/questions/23668427/pandas-joining-multiple-dataframes-on-columns?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
6. https://regexone.com/lesson/conditionals
7. https://stackoverflow.com/questions/13413590/how-to-drop-rows-of-pandas-dataframe-whose-value-in-certain-columns-is-nan?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
8. http://jonathansoma.com/lede/foundations-2017/classes/pandas-text-part-1/classwork/
https://stackoverflow.com/questions/25351968/how-to-display-full-non-truncated-dataframe-information-in-html-when-convertin?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
https://regex101.com/
https://stackoverflow.com/questions/9012008/pythons-re-return-true-if-regex-contains-in-the-string?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
https://stackoverflow.com/questions/37725195/pandas-replace-values-based-on-index?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
https://docs.python.org/3/library/re.html#match-objects
https://stackoverflow.com/questions/33005872/seaborn-regplot-partially-see-through-alpha?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
https://github.com/mwaskom/seaborn/issues/202
https://seaborn.pydata.org/generated/seaborn.lmplot.html
https://stackoverflow.com/questions/17071871/select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa